In [1]:
import numpy as np
from heapq import *
import time
import pandas as pd
# loaddata
T=pd.read_csv("C:\\Users\\wesle\\Desktop\\contest\\data\\taozongyong.csv")

In [3]:
T.describe()# xid 1-548 yid 1-421 hour 3-20 day 1-5

,Unnamed: 0,xid,yid,date_id,hour,model,wind
count,2.076372e+07,2.076372e+07,2.076372e+07,2.076372e+07,2.076372e+07,20763720.0,2.076372e+07
mean,1.038186e+08,2.745000e+02,2.110000e+02,8.000000e+00,1.150000e+01,1.0,1.783058e+01
std,5.993970e+07,1.581937e+02,1.215319e+02,1.414214e+00,5.188128e+00,0.0,7.285068e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00,3.000000e+00,1.0,0.000000e+00
25%,5.190930e+07,1.377500e+02,1.060000e+02,7.000000e+00,7.000000e+00,1.0,1.310000e+01
50%,1.038186e+08,2.745000e+02,2.110000e+02,8.000000e+00,1.150000e+01,1.0,1.730000e+01
75%,1.557279e+08,4.112500e+02,3.160000e+02,9.000000e+00,1.600000e+01,1.0,2.210000e+01
max,2.076372e+08,5.480000e+02,4.210000e+02,1.000000e+01,2.000000e+01,1.0,5.280000e+01


In [53]:
import os
citypath = os.path.expanduser('~wesle\Desktop\contest\data\CityData.csv')
city=pd.read_csv(citypath)
city 

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


In [59]:
def heuristic(a, b):
    try:
        c=abs(a[2]-b[2])
    except Exception as e:
        c=0
    manhattandis=abs(b[0] - a[0])+abs(b[1] - a[1])
    return (manhattandis)

def truedis(a,b):
    return (abs(a[2]-b[2]))

def astar(array, start, goal):
    
    neighbors = [(0,1,1),(0,-1,1),(1,0,1),(-1,0,1),(0,0,1)]

    close_set = set()
    came_from = {}
    gscore = {start:0} # g(n)
    fscore = {start:heuristic(start, goal)} # h(n)
    oheap = [] #openset each time we check

    heappush(oheap, (fscore[start], start))# first push start into the openset,each node store tuple and its fscore
 
    timeout = time.time() + 400  # 400s per loop
    
    
    while oheap:
        
        current = heappop(oheap)[1]
#         print(current)
        if time.time()>timeout:
            print("time out")
            return "time out"
        if current[:2] == goal[:2]:
            print("we get it")
            data = []
            while current in came_from:
                data.append(current)
                current = came_from[current]
            
            return data

        close_set.add(current)
        for i, j, k in neighbors:
            neighbor = current[0] + i, current[1] + j,current[2]+ k
            # if we need to speed up ,set alpha to be greater than 1
            tentative_g_score = gscore[current] + 1* truedis(current, neighbor)
            
            if 0 <= neighbor[0] < array.shape[0]:
                if 0 <= neighbor[1] < array.shape[1]:
                    if 0<=neighbor[2]<= goal[2]:
                        if array[neighbor[0]][neighbor[1]][neighbor[2]] == 1: # danger zone
                            continue
                    else:
                        # array bound t upperlimit
                        continue
                else:
                    # array bound y walls
                    continue
            else:
                # array bound x walls
                continue
                
            if neighbor in close_set and tentative_g_score >= gscore.get(neighbor, 0):# if this one is visited and 
                continue
                
            if  tentative_g_score < gscore.get(neighbor, 0) or neighbor not in [i[1]for i in oheap]: # this neighbour not in the heap
                came_from[neighbor] = current
                gscore[neighbor] = tentative_g_score
                fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heappush(oheap, (fscore[neighbor], neighbor))
                
    return False

In [60]:
def initegraph(df,day):
    thismap=np.zeros((548,421,18*30)) # we set all values to zero 
    # the index is from 0 to 547   0 to 18*30 (18 hours in total)
    df=df[df.date_id==day]
    zone=df[df.wind>=15][["xid","yid","hour"]] # we subset the dataset which contains tuples with wind is greater than 15
    
    for x,y,hour in zone.values:
        thismap[x-1][y-1][(hour-3)*30:(hour-2)*30]=1 # we set the map  since the index is less than one
        # the hour is start form 3 to 21  : if hour is 3 which means t start from 0 to 29 should be set to one  so it should be 0:30
    return thismap
    

In [201]:
city[0:4]

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234


In [61]:
npmap=initegraph(T,6) # construct the map for dayone

In [26]:
city

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


In [62]:
astar(npmap,(142-1,328-1,0),(84-1,203-1,18*30-1)) # it seems that is miss the starting point (141,327,0)

we get it


[(83, 202, 195),
 (82, 202, 194),
 (81, 202, 193),
 (80, 202, 192),
 (79, 202, 191),
 (78, 202, 190),
 (77, 202, 189),
 (77, 203, 188),
 (77, 204, 187),
 (77, 205, 186),
 (77, 206, 185),
 (77, 207, 184),
 (77, 208, 183),
 (77, 209, 182),
 (77, 210, 181),
 (77, 211, 180),
 (77, 212, 179),
 (77, 213, 178),
 (77, 214, 177),
 (77, 215, 176),
 (77, 216, 175),
 (77, 217, 174),
 (77, 218, 173),
 (77, 219, 172),
 (77, 220, 171),
 (77, 221, 170),
 (77, 222, 169),
 (77, 223, 168),
 (77, 224, 167),
 (77, 225, 166),
 (77, 226, 165),
 (77, 227, 164),
 (77, 228, 163),
 (77, 229, 162),
 (77, 230, 161),
 (77, 231, 160),
 (77, 232, 159),
 (77, 233, 158),
 (77, 234, 157),
 (77, 235, 156),
 (77, 236, 155),
 (77, 237, 154),
 (77, 238, 153),
 (77, 239, 152),
 (77, 240, 151),
 (77, 241, 150),
 (77, 242, 149),
 (77, 243, 148),
 (77, 244, 147),
 (77, 245, 146),
 (77, 246, 145),
 (77, 247, 144),
 (77, 248, 143),
 (77, 249, 142),
 (77, 250, 141),
 (77, 251, 140),
 (77, 252, 139),
 (77, 253, 138),
 (77, 254, 137

In [63]:
def findpath(thismap,balloon,city):
    startcity=tuple(city.iloc[0,1:])
    start=(startcity[0]-1,startcity[1]-1,0)
    endcity=tuple(city.iloc[balloon,1:])
    maxmin=18*30-1
    end=(endcity[0]-1,endcity[1]-1,maxmin)
    return astar(thismap,start,end)

In [64]:
def findall(city,df):
    # input : city --- loaction of each city
    #         df ---- contain the forecast of the weather
    # output: dictionary key(day,destination): list of tuple (path) 
    solution=dict()
    for date in range(6,11):
        mapthisday=initegraph(df,date)
        for citycode in range(1,11):
            print("Day {} Hey we are going to find the path for the no.{}".format(date,citycode))
            solution[(date,citycode)]=findpath(mapthisday,citycode,city)
    return solution
    
    

In [65]:
mydic=findall(city,T)

Day 6 Hey we are going to find the path for the no.1
we get it
Day 6 Hey we are going to find the path for the no.2
we get it
Day 6 Hey we are going to find the path for the no.3
we get it
Day 6 Hey we are going to find the path for the no.4
time out
Day 6 Hey we are going to find the path for the no.5
time out
Day 6 Hey we are going to find the path for the no.6
time out
Day 6 Hey we are going to find the path for the no.7
time out
Day 6 Hey we are going to find the path for the no.8
time out
Day 6 Hey we are going to find the path for the no.9
we get it
Day 6 Hey we are going to find the path for the no.10
we get it
Day 7 Hey we are going to find the path for the no.1
time out
Day 7 Hey we are going to find the path for the no.2
we get it
Day 7 Hey we are going to find the path for the no.3
time out
Day 7 Hey we are going to find the path for the no.4
time out
Day 7 Hey we are going to find the path for the no.5
time out
Day 7 Hey we are going to find the path for the no.6
time out
D

In [66]:
import pickle
path="C:\\Users\\wesle\\Desktop\\contest\\data\\solution"
with open(path,"bw") as f:
    pickle.dump(mydic,f)

In [67]:
mydic[(10,10)]

[(188, 273, 103),
 (187, 273, 102),
 (186, 273, 101),
 (185, 273, 100),
 (184, 273, 99),
 (183, 273, 98),
 (182, 273, 97),
 (181, 273, 96),
 (180, 273, 95),
 (179, 273, 94),
 (178, 273, 93),
 (177, 273, 92),
 (176, 273, 91),
 (175, 273, 90),
 (174, 273, 89),
 (173, 273, 88),
 (172, 273, 87),
 (171, 273, 86),
 (170, 273, 85),
 (169, 273, 84),
 (168, 273, 83),
 (167, 273, 82),
 (166, 273, 81),
 (165, 273, 80),
 (165, 274, 79),
 (165, 275, 78),
 (165, 276, 77),
 (165, 277, 76),
 (165, 278, 75),
 (165, 279, 74),
 (165, 280, 73),
 (165, 281, 72),
 (165, 282, 71),
 (165, 283, 70),
 (165, 284, 69),
 (165, 285, 68),
 (165, 286, 67),
 (165, 287, 66),
 (165, 288, 65),
 (165, 289, 64),
 (165, 290, 63),
 (165, 291, 62),
 (165, 292, 61),
 (165, 293, 60),
 (165, 294, 59),
 (165, 295, 58),
 (165, 296, 57),
 (165, 297, 56),
 (165, 298, 55),
 (165, 299, 54),
 (165, 300, 53),
 (165, 301, 52),
 (165, 302, 51),
 (165, 303, 50),
 (165, 304, 49),
 (165, 305, 48),
 (165, 306, 47),
 (165, 307, 46),
 (165, 308

In [1]:
import pickle
path="C:\\Users\\wesle\\Desktop\\contest\\data\\solution"


In [2]:
with open(path,"br") as f:
    mydic=pickle.load(f)

In [15]:
mydic[(6,5)]=="time out"

True

In [51]:
import pandas as pd
a=pd.DataFrame([[1,1,1,1,1]]*2,columns=["destination","date","time","xid","yid"])
a

,destination,date,time,xid,yid
0,1,1,1,1,1
1,1,1,1,1,1


In [48]:
b=pd.DataFrame([[1,1,1,1,1]],columns=["destination","date","time","xid","yid"])
a=a.append(b)
a

,destination,date,time,xid,yid
0,1,1,1,1,1
0,1,1,1,1,1


In [45]:
a.to_csv("temp.csv")

In [73]:
city

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


In [74]:
city.get_value(1,"xid")

84

In [169]:
time = pd.date_range('03:00',periods=20,freq='2min')
time= pd.DataFrame(time.strftime("%H:%M"))
time

,0
0,03:00
1,03:02
2,03:04
3,03:06
4,03:08
5,03:10
6,03:12
7,03:14
8,03:16
9,03:18


In [132]:
length=len(a)
b=pd.DataFrame([[1,2] for i in range(length)],columns=["destination","date"])
b

,destination,date
0,1,2
1,1,2
2,1,2
3,1,2
4,1,2


In [135]:
c=pd.DataFrame([[3,4] for i in range(length)],columns=["destination","date"])

In [152]:
pd.concat([b,a,c],axis=1,ignore_index=False)

,destination,date,0
0,1,2,03:00
1,1,2,03:02
2,1,2,03:04
3,1,2,03:06
4,1,2,03:08


In [147]:
# c.rename(str,axis=1)
c.rename(index=str, columns={"destination": "a", "B": "c"})

,a,date
0,3,4
1,3,4
2,3,4
3,3,4
4,3,4


In [149]:
c=pd.DataFrame()

In [151]:
pd.concat([c,a],axis=0)

,0
0,03:00
1,03:02
2,03:04
3,03:06
4,03:08


In [173]:
mydic[(6,1)][:5]

[(83, 202, 195),
 (82, 202, 194),
 (81, 202, 193),
 (80, 202, 192),
 (79, 202, 191)]

In [182]:
mydic[(6,1)][:5][::-1]

[(79, 202, 191),
 (80, 202, 192),
 (81, 202, 193),
 (82, 202, 194),
 (83, 202, 195)]

In [208]:
temp=pd.DataFrame([(x+1,y+1) for x ,y,t in mydic[(6,1)][:5]],columns=["3","4"])
temp

,3,4
0,84,203
1,83,203
2,82,203
3,81,203
4,80,203


In [214]:
a=[1,1,4]
a.append((1,3))
a

[1, 1, 4, (1, 3)]

In [216]:
final=pd.DataFrame()
for date,balloonid in mydic.keys():
    # we didn't find the path
    if mydic[(date,balloonid)]=="time out":
#         x=city.get_value(balloonid,"xid")
#         y=city.get_value(balloonid,"yid")
#         time = pd.date_range('03:00','21:00',freq='2min')
#         time = pd.DataFrame(time.strftime("%H:%M"),columns=["time"])
#         df1=pd.DataFrame([[balloonid,date] for i in range(len(time))],columns=["destination","date"])
#         df2=pd.DataFrame([[x,y] for i in range(len(time))],columns=["xid","yid"])
#         this_route=pd.concat([df1,time,df2],axis=1,ignore_index=False)
#         final=pd.concat([final,this_route],axis=0)
         pass
    else:
        length=len(mydic[(date,balloonid)])+1
        time = pd.date_range('03:00',periods=length,freq='2min')
        x=city.get_value(balloonid,"xid")
        y=city.get_value(balloonid,"yid")
        df1=pd.DataFrame([[balloonid,date] for i in range(length)],columns=["destination","date"])
        idlist=[(x+1,y+1) for x,y,t in mydic[(date,balloonid)]]
        idlist.append((142,328)) # add the starting point
        idlist=idlist[::-1] # reverse the list 
        df2=pd.DataFrame(idlist,columns=["xid","yid"])# push it into dataframe
        time = pd.DataFrame(time.strftime("%H:%M"),columns=["time"])
#         print(time)
        this_route=pd.concat([df1,time,df2],axis=1,ignore_index=False)
#         print(this_route)
        final=pd.concat([final,this_route],axis=0)
    

In [217]:
final.to_csv("submission_1_24_15.csv")

In [181]:
a=["1","3","4"]
a[::-1]

['4', '3', '1']